In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score
import re
from cleantext import clean
import string
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import losses
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


In [2]:
df=pd.read_excel('Copy of Bot Data.xlsx')
#convert text to features 

#convert text to lower case 
df['biography']=df['biography'].str.lower()
df['comment']=df['comment'].str.lower()

#expand contractions 
contractions_dict = {"aren't": "are not", "don't": "do not", "Don't": "do not", "I'm": "I am", "i'm": "I am", 
                    "it's": "it is", "y'all": "you all", "Y'all": "you all", "didn't": "did not", "won't": "will not",
                   "I'll": "I will", "i'll": "I will", "can't": "can not"}
contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))

def expand_contractions(text, expand_dict):
    def replace(match):
        return expand_dict[match.group(0)]
    return contractions_re.sub(replace, text)
df['biography']=df['biography'].apply(lambda x:expand_contractions(x, contractions_dict ))
df['comment']=df['comment'].apply(lambda x:expand_contractions(x, contractions_dict))

#remove punctuation (%, ', !)
df['biography'] = df['biography'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '' , x))
df['comment'] = df['comment'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '' , x))

bios=df['biography'].astype(str)
comments=df['comment']




In [3]:
#filter out emojis 
new_bios=[]
for b in bios: 
    temp=clean(b, no_emoji=True)
    temp=temp.replace("\n", " ")
    if temp=='':
        new_bios.append('None')
    else: 
        new_bios.append(temp)
new_comments=[]
for c in comments:
    temp=clean(c, no_emoji=True)
    temp=temp.replace("\n", " ")
    if temp=='':
        new_comments.append('None')
    else: 
        new_comments.append(temp)
df_filtered=df.copy()
df_filtered['biography']=new_bios
df_filtered['comment']=new_comments

In [4]:
#combine text into one string 
df_filtered['combined']=df_filtered['biography'] + '' + df_filtered['comment']

#train and test set split 
train_set, test_set= train_test_split(
df_filtered, test_size=.2, random_state=0)
from sklearn.feature_extraction.text import CountVectorizer

vectorizer1 = CountVectorizer(min_df=0, lowercase=False)
vectorizer1.fit(train_set['combined'])
text_array = vectorizer1.transform(train_set['combined']).toarray()
text_df = pd.DataFrame(data=text_array,columns = vectorizer1.get_feature_names())
text_df
#can't apply the transform to separate columns because that leads to duplicates 

,09,10,100,1000,10000,100k,10k,12,12600,127491,...,youtubechcd3chxjkthat,youtubecomchannelucokk65muctgbypbguysemathis,youtubecomchannelucssxa8e6u1cvojcvelxugw,yt,zainab,zeldon,zero,zilahi,zooming,zu
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
247,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
248,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
249,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
#variables are binary so don't need to be normalized
#apply a low variance filter 
cols= text_df.columns
variance1 = text_df.var()
filtered_cols1 = [ ]
for i in range(0,len(variance1)):
    if variance1[i]>=0.006: #threshold for variance is 1%, don't want to include constant variables
        filtered_cols1.append(cols[i])
# filtered_cols = cols, so none of the remaining columns are constant with a variance of 0
text_df

,09,10,100,1000,10000,100k,10k,12,12600,127491,...,youtubechcd3chxjkthat,youtubecomchannelucokk65muctgbypbguysemathis,youtubecomchannelucssxa8e6u1cvojcvelxugw,yt,zainab,zeldon,zero,zilahi,zooming,zu
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
247,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
248,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
249,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
#select features: goes from 1810 to 473! 
text_df=text_df[filtered_cols1]

In [7]:
X_train=text_df
Y_train=train_set['bot classification (0-not a bot, 1-bot)']
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(X_train, Y_train)
text_classifier=classifier.predict(X_train)

In [8]:
#transforming bio and comments separately
vectorizer2 = CountVectorizer(min_df=0, lowercase=False)
vectorizer3 = CountVectorizer(min_df=0, lowercase=False)
vectorizer2.fit(train_set['biography'])
vectorizer3.fit(train_set['comment'])
bio_array = vectorizer2.transform(train_set['biography']).toarray()
bio_df = pd.DataFrame(data=bio_array,columns = vectorizer2.get_feature_names())
comment_array = vectorizer3.transform(train_set['comment']).toarray()
comment_df = pd.DataFrame(data=comment_array,columns = vectorizer3.get_feature_names())

In [9]:
#filter bio and comment features
cols= bio_df.columns
variance1 = bio_df.var()
filtered_cols2 = [ ]
for i in range(0,len(variance1)):
    if variance1[i]>=0.006: #threshold for variance is 1%, don't want to include constant variables
        filtered_cols2.append(cols[i])
bio_df=bio_df[filtered_cols2]

cols= comment_df.columns
variance1 = comment_df.var()
filtered_cols3 = [ ]
for i in range(0,len(variance1)):
    if variance1[i]>=0.006: #threshold for variance is 1%, don't want to include constant variables
        filtered_cols3.append(cols[i])
comment_df=comment_df[filtered_cols3]

In [10]:
#classify using bio df
from sklearn.linear_model import LogisticRegression
classifier2 = LogisticRegression()
classifier2.fit(bio_df, Y_train)
bio_classifier=classifier2.predict(bio_df)

#classify using comment df 
classifier3 = LogisticRegression()
classifier3.fit(comment_df, Y_train)
comment_classifier=classifier3.predict(comment_df)

In [11]:
#merge bio and comment dataframes to output one feature 
#this uses a different count vector for bio and comments, unlike the first classifier that concatenated 
#the two into one string 

df_combined=pd.DataFrame()
df_combined=pd.concat([comment_df, bio_df],axis=1)
classifier4 = LogisticRegression()
classifier4.fit(df_combined, Y_train)
text_classifier2=classifier4.predict(df_combined)


In [12]:

#normalize the training dataset features (followers, following, etc)
train_set_filtered=train_set[['Username', 'Follower Count', 'Following Count',
       'Follower/Following Ratio', 'Number of posts', '# of likes on comment ', 'time of comment after post (minutes)', 'bot classification (0-not a bot, 1-bot)']]
df_normalized=train_set_filtered.copy()
numerical_cols=['Follower Count', 'Following Count',
       'Follower/Following Ratio', 'Number of posts', '# of likes on comment ', 'time of comment after post (minutes)']
for c in numerical_cols:
    df_normalized[c]= (train_set_filtered[c] - train_set_filtered[c].min()) / (train_set_filtered[c].max() - train_set_filtered[c].min())
df_normalized=df_normalized.set_index('Username')

#normalize test set 
test_normalized=test_set.copy()
test_normalized=test_normalized[['Username','Follower Count', 'Following Count',
       'Follower/Following Ratio', 'Number of posts', '# of likes on comment ', 'time of comment after post (minutes)', 'bot classification (0-not a bot, 1-bot)']]
for c in numerical_cols: 
    test_normalized[c]= (test_normalized[c] - test_normalized[c].min()) / (test_normalized[c].max() - test_normalized[c].min())  
test_normalized=test_normalized.set_index('Username')
    

In [13]:
#try text classifying as a feature itself- vocabulary generated grouping bio and comments together 
X_train1=df_normalized.copy()
X_train1['text_classifier']=text_classifier
Y_train=X_train1['bot classification (0-not a bot, 1-bot)']
X_train1=X_train1.drop(['bot classification (0-not a bot, 1-bot)'], axis=1)

test_set['combined']=test_set['biography'] + '' + test_set['comment']
text_array_test = vectorizer1.transform(test_set['combined']).toarray()
text_df_test = pd.DataFrame(data=text_array_test,columns = vectorizer1.get_feature_names())
text_df_test=text_df_test[filtered_cols1]
text_classifier_test=classifier.predict(text_df_test)
X_test1=test_normalized.copy()
X_test1['text_classifier']=text_classifier_test
Y_test=X_test1['bot classification (0-not a bot, 1-bot)']
X_test1=X_test1.drop(['bot classification (0-not a bot, 1-bot)'], axis=1)
final_classifier1 = LogisticRegression()
final_classifier1.fit(X_train1, Y_train)
score = final_classifier1.score(X_test1, Y_test)
print("Accuracy:", score)

Accuracy: 0.873015873015873


<ipython-input-13-dbe5ba1e6b8a>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_set['combined']=test_set['biography'] + '' + test_set['comment']


In [14]:
#separate text classifying into two features: bio and comments 
X_train2=df_normalized.copy()
X_train2['comment_classifier']=comment_classifier
X_train2['bio_classifier']=bio_classifier
X_train2=X_train2.drop(['bot classification (0-not a bot, 1-bot)'], axis=1)

bio_array_test = vectorizer2.transform(test_set['biography']).toarray()
bio_df_test = pd.DataFrame(data=bio_array_test,columns = vectorizer2.get_feature_names())
comment_array_test = vectorizer3.transform(test_set['comment']).toarray()
comment_df_test = pd.DataFrame(data=comment_array_test,columns = vectorizer3.get_feature_names())
bio_df_test=bio_df_test[filtered_cols2]
comment_df_test=comment_df_test[filtered_cols3]
X_test2=test_normalized.copy()
bio_classifier_test=classifier2.predict(bio_df_test)
comment_classifier_test=classifier3.predict(comment_df_test)
X_test2['comment_classifier']=comment_classifier_test
X_test2['bio_classifier']=bio_classifier_test
X_test2=X_test2.drop(['bot classification (0-not a bot, 1-bot)'], axis=1)
final_classifier2 = LogisticRegression()
final_classifier2.fit(X_train2, Y_train)
score2 = final_classifier2.score(X_test2, Y_test)
print("Accuracy:", score2)

Accuracy: 0.8888888888888888


In [15]:
##try text classifying as ONE feature- vocabulary generated separately for bio and comments 
X_train3=df_normalized.copy()
X_train3['text_classifier']=text_classifier2
X_train3=X_train3.drop(['bot classification (0-not a bot, 1-bot)'], axis=1)

df_combined_test=pd.concat([comment_df_test, bio_df_test],axis=1)
X_test3=test_normalized.copy()
text_classifier2_test=classifier4.predict(df_combined_test)
X_test3['textclassifier']=text_classifier2_test

X_test3=X_test3.drop(['bot classification (0-not a bot, 1-bot)'], axis=1)
final_classifier3 = LogisticRegression()
final_classifier3.fit(X_train3, Y_train)
score3 = final_classifier3.score(X_test3, Y_test)
print("Accuracy:", score3)

Accuracy: 0.8888888888888888
